In [1]:
trait Monoid[A]:
    def combine(a1: A, a2: A): A
    def empty: A

defined trait Monoid

#### Exercise 10.1

Give `Monoid` instances for integer addition and multiplication as well as the `Boolean` operators.

In [2]:
val intAddition: Monoid[Int] = new:
    def combine(a1: Int, a2: Int) = a1 + a2
    def empty = 0

val intMultiplication: Monoid[Int] = new:
    def combine(a1: Int, a2: Int) = a1 * a2
    def empty = 1

val booleanOr: Monoid[Boolean] = new:
    def combine(a1: Boolean, a2: Boolean) = a1 || a2
    def empty = false

val booleanAnd: Monoid[Boolean] = new:
    def combine(a1: Boolean, a2: Boolean) = a1 && a2
    def empty = true

intAddition: Monoid[Int] = ammonite.$sess.cmd1$$anon$1@3d1aa190
intMultiplication: Monoid[Int] = ammonite.$sess.cmd1$$anon$2@52234cd8
booleanOr: Monoid[Boolean] = ammonite.$sess.cmd1$$anon$3@1f2d7e59
booleanAnd: Monoid[Boolean] = ammonite.$sess.cmd1$$anon$4@493f2e36

#### Exercise 10.2

Give a Monoid instance for combining `Option` values.

In [3]:
def optionMonoid[A]: Monoid[Option[A]] = new:
    def combine(a1: Option[A], a2: Option[A]): Option[A] = a1.orElse(a2)
    def empty = None

defined function optionMonoid

#### Exercise 10.3

A function having the same argument and return type is sometimes called an `endofunction`. Write a monoid for endofunctions.


In [4]:
def endoMonoid[A]: Monoid[A => A] = new:
    def combine(f1: A => A, f2: A => A): A => A = f1 compose f2
    def empty = identity

defined function endoMonoid

#### Exercise 10.4

Use the property-based testing framework we developed in part 2 to implement a property for the monoid laws. Use your property to test the monoids we’ve written.

In [5]:
import $file.scripts.testing, testing.*

Compiling /Users/galex/src/github.com/galexy/fps2/scripts/testing.sc

import $file.$              , testing.*


In [8]:
def monoidLaws[A](m: Monoid[A], gen: Gen[A]): Prop =
    val trips = gen ** gen ** gen
    val p1 = forAll(trips) { case ((x, y), z) => 
        m.combine(m.combine(x, y), z) == m.combine(x, m.combine(y, z))
    }

    val p2 = forAll(gen)(x => m.combine(x, m.empty) == m.combine(m.empty, x))

    p1 && p2

defined function monoidLaws

In [11]:
monoidLaws(booleanOr, boolean).run()
monoidLaws(booleanAnd, boolean).run()
monoidLaws(intAddition, choose(Int.MinValue, Int.MaxValue)).run()
monoidLaws(intMultiplication, choose(Int.MinValue, Int.MaxValue)).run()


+ OK, passed 100 tests.
+ OK, passed 100 tests.
+ OK, passed 100 tests.
+ OK, passed 100 tests.


#### Exercise 10.5

Implement `foldMap`.

In [9]:
def foldMap[A, B](as: List[A], m: Monoid[B])(f: A => B): B =
    as.map(f).foldLeft(m.empty)(m.combine)

defined function foldMap

#### Exercise 10.6

*Hard*: The `foldMap` function can be implemented using either `foldLeft` or `foldRight`. But you can also write `foldLeft` and `foldRight` using `foldMap`! Try it.

In [74]:
def cons[B](combiner: (B, B) => B, init: B): Monoid[B] = new:
    def combine(acc: B, b: B): B = combiner
    def empty = init

def foldLeft2[A, B](as: List[A])(init: B)(f: (B, A) => B): B =
    

#### Exercise 10.7

Implement a `foldMap` for `IndexedSeq`. Your implementation should use the strategy of splitting the sequence in two, recursively processing each half, and then adding the answers together with the monoid.

In [31]:
def foldMapV[A, B](as: IndexedSeq[A], m: Monoid[B])(f: A => B): B =
    if (as.length == 0)
        m.empty
    else if (as.length == 1)
        f(as(0))
    else
        val (left, right) = as.splitAt(as.length / 2)
        m.combine(foldMapV(left, m)(f), foldMapV(right, m)(f))

defined function foldMapV

In [32]:
foldMapV(Array(1, 2, 3, 4, 5), intAddition)(identity)
foldMapV(Array(): Array[Int], intAddition)(identity)

res31_0: Int = 15
res31_1: Int = 0

#### Exercise 10.9

*Hard*: Use `foldMap` to detect whether a given `IndexedSeq[Int]` is ordered. You’ll need to come up with a creative `Monoid`.


In [33]:
enum IsOrdered:
    case NotOrdered
    case EmptyOrdered
    case Ordered(start: Int, end: Int)

val IsOrderedMonoid: Monoid[IsOrdered] = new:
    import IsOrdered.*

    def combine(left: IsOrdered, right: IsOrdered): IsOrdered = (left, right) match
        case (Ordered(s1, e1), Ordered(s2, e2)) if e1 < s2 => Ordered(s1, e2)
        case (o1@Ordered(_, _), EmptyOrdered) => o1
        case (EmptyOrdered, o2@(Ordered(_, _))) => o2
        case _ => NotOrdered
    
    def empty = EmptyOrdered

def isOrdered(is: IndexedSeq[Int]): Boolean =
    import IsOrdered.*

    foldMapV(is, IsOrderedMonoid)(i => Ordered(i, i)) != NotOrdered

defined class IsOrdered
IsOrderedMonoid: Monoid[IsOrdered] = ammonite.$sess.cmd32$$anon$1@20517810
defined function isOrdered

In [34]:
isOrdered(Array(1, 2, 3, 4))
isOrdered(Array(1, 2, 4, 3, 5))
isOrdered(Array():Array[Int])

res33_0: Boolean = true
res33_1: Boolean = false
res33_2: Boolean = true

#### Exercise 10.10

Write a monoid instance for `WC` and make sure that it meets the monoid laws.

In [44]:
enum WC:
    case Stub(chars: String)
    case Part(lStub: String, words: Int, rStub: String)

val wcMonoid: Monoid[WC] = new:
    import WC.*
    def combine(left: WC, right: WC): WC = (left, right) match
        case (Stub(l), Stub(r)) => Stub(l + r)
        case (Stub(l), Part(lStub, words, rStub)) => Part(l + lStub, words, rStub)
        case (Part(lStub, words, rStub), Stub(r)) => Part(lStub, words, rStub + r)
        case (Part(llStub, lWords, lrStub), Part(rlStub, rWords, rrStub)) =>
            Part(llStub, lWords + rWords + (if (lrStub + rlStub).isEmpty then 0 else 1), rrStub)
    def empty = Stub("")


defined class WC
wcMonoid: Monoid[WC] = ammonite.$sess.cmd43$$anon$1@22da2684

#### Exercise 10.11

Use the `WC` monoid to implement a function that counts words in a String by recursively splitting it into substrings and counting the words in those substrings.

In [37]:
def wordCount(text: String): Int =
    def fold(s: String): WC =
        if (s.length == 0)
            wcMonoid.empty
        else if (s.length == 1)
            WC.Stub(s)
        else
            val (left, right) = s.splitAt(s.length / 2)
            wcMonoid.combine(fold(left), fold(right))
    fold(text) match
        case WC.Stub(c) => if c.length > 0 then 1 else 0
        case WC.Part(l, w, r) => (if l.length > 0 then 1 else 0) + w + (if r.length > 0 then 1 else 0)
    

defined function wordCount

In [43]:
wordCount("")
wordCount("f")
wordCount("f f")

res42_0: Int = 0
res42_1: Int = 1
res42_2: Int = 1

In [46]:
object Monoid:
    given intAddition: Monoid[Int] with
        def combine(a1: Int, a2: Int) = a1 + a2
        val empty = 0

    val intMultiplication: Monoid[Int] = new:
        def combine(a1: Int, a2: Int) = a1 * a2
        val empty = 1

    given string: Monoid[String] with
        def combine(a1: String, a2: String) = a1 + a2
        val empty = ""

    val booleanOr: Monoid[Boolean] = new:
        def combine(x: Boolean, y: Boolean) = x || y
        val empty = false

    val booleanAnd: Monoid[Boolean] = new:
        def combine(x: Boolean, y: Boolean) = x && y
        val empty = true

    given endoMonoid[A]: Monoid[A => A] = new:
        def combine(f1: A => A, f2: A => A): A => A = f1 compose f2
        def empty = identity 

    def dual[A](m: Monoid[A]): Monoid[A] = new:
        def combine(x: A, y: A): A = m.combine(y, x)
        val empty = m.empty

defined object Monoid

#### Exercise 10.12

Implement `Foldable[List]`, `Foldable[IndexedSeq]`, and `Foldable[LazyList]`. Remember that `foldRight`, `foldLeft`, and `foldMap` can all be implemented in terms of each other, but that might not be the most efficient implementation.


In [52]:
trait Foldable[F[_]]:
    import Monoid.*

    extension [A](as: F[A])
        def foldR[B](acc: B)(f: (A, B) => B): B = 
            as.foldMap(f.curried)(using dual(endoMonoid))(acc)

        def foldL[B](acc: B)(f: (B, A) => B): B =
            as.foldMap(a => b => f(b, a))(using endoMonoid)(acc)

        def foldMap[B](f: A => B)(using m: Monoid[B]): B =
            as.foldL(m.empty)((b, a) => m.combine(b, f(a)))

        def combineAll(using m: Monoid[A]): A = 
            as.foldL(m.empty)(m.combine)

defined trait Foldable

In [75]:
given Foldable[List] with
    extension [A](as: List[A])
        override def foldR[B](acc: B)(f: (A, B) => B): B = as.foldRight(acc)(f)
        override def foldL[B](acc: B)(f: (B, A) => B): B = as.foldLeft(acc)(f)

given Foldable[IndexedSeq] with
    extension [A](as: IndexedSeq[A])
        override def foldR[B](acc: B)(f: (A, B) => B): B = as.foldRight(acc)(f)
        override def foldL[B](acc: B)(f: (B, A) => B): B = as.foldLeft(acc)(f)
        override def foldMap[B](f: A => B)(using mb: Monoid[B]): B =
            foldMapV(as, mb)(f)
        

defined object 
defined object 

In [59]:
def foo(is: List[Int])(using Foldable[List]): Int =
    import Monoid.intMultiplication
    is.foldMap(_ + 1)(using intMultiplication)

foo(List(1, 2, 3))

defined function foo
res58_1: Int = 24

#### Exercise 10.13

Recall the binary Tree data type from chapter 3. Implement a Foldable instance for it.

In [61]:
enum Tree[+A]:
    case Leaf(value: A)
    case Branch(left: Tree[A], right: Tree[A])

given Foldable[Tree] with
    import Tree.*
    extension [A](as: Tree[A])
        override def foldR[B](acc: B)(f: (A, B) => B): B = as match
            case Leaf(a) => f(a, acc)
            case Branch(l, r) => l.foldR(r.foldR(acc)(f))(f)

        override def foldL[B](acc: B)(f: (B, A) => B): B = as match
            case Leaf(a) => f(acc, a)
            case Branch(l, r) => r.foldL(l.foldL(acc)(f))(f)

        override def foldMap[B](f: A => B)(using m: Monoid[B]): B = as match
            case Leaf(a) => f(a)
            case Branch(l, r) => m.combine(l.foldMap(f), r.foldMap(f))


defined class Tree
defined object 

#### Exercise 10.14

Write a `Foldable[Option]` instance.

In [62]:
given Foldable[Option] with
    extension [A](as: Option[A])
        override def foldR[B](acc: B)(f: (A, B) => B): B = as match
            case None => acc
            case Some(a) => f(a, acc)
        override def foldL[B](acc: B)(f: (B, A) => B): B = as match
            case None => acc
            case Some(a) => f(acc, a)
        override def foldMap[B](f: A => B)(using m: Monoid[B]): B = as match
            case None => m.empty
            case Some(a) => f(a)

defined object 

Exercise 10.15

Any `Foldable` structure can be turned into a `List`. Add a `toList` extension method to the `Foldable` trait and provide a concrete implementation in terms of the other methods on `Foldable`.

In [63]:
trait Foldable[F[_]]:
    import Monoid.*

    extension [A](as: F[A])
        def foldR[B](acc: B)(f: (A, B) => B): B = 
            as.foldMap(f.curried)(using dual(endoMonoid))(acc)

        def foldL[B](acc: B)(f: (B, A) => B): B =
            as.foldMap(a => b => f(b, a))(using endoMonoid)(acc)

        def foldMap[B](f: A => B)(using m: Monoid[B]): B =
            as.foldL(m.empty)((b, a) => m.combine(b, f(a)))

        def combineAll(using m: Monoid[A]): A = 
            as.foldL(m.empty)(m.combine)

        def toList: List[A] = foldR(Nil: List[A])(_ :: _)

defined trait Foldable

#### Exercise 10.16

Implement `productMonoid` using a `ma: Monoid[A]` and `mb: Monoid[B]`. Notice that your implementation of combine is associative so long as `ma.combine` and `mb.combine` are both associative.


In [64]:
given productMonoid[A, B](using ma: Monoid[A], mb: Monoid[B]): Monoid[(A, B)] with
    def combine(x: (A, B), y: (A, B)) = (ma.combine(x(0), y(0)), mb.combine(x(1), y(1)))
    val empty = (ma.empty, mb.empty)

defined class productMonoid

given mapMergeMonoid[K, V](using )

In [65]:
given mapMergeMonoid[K, V](using m: Monoid[V]): Monoid[Map[K, V]] with
    def combine(a: Map[K, V], b: Map[K, V]): Map[K, V] =
        (a.keySet ++ b.keySet).foldLeft(empty){ (acc, k) =>
            acc.updated(k, m.combine(a.getOrElse(k, m.empty),
                                     b.getOrElse(k, m.empty)))
        }

    val empty = Map()

defined class mapMergeMonoid

In [67]:
val a = Map("k1" -> 1, "k2" -> 2)
val b = Map("k2" -> 3, "k3" -> 4)
import Monoid.intAddition
val m = summon[Monoid[Map[String, Int]]]
m.combine(a, b)

a: Map[String, Int] = Map("k1" -> 1, "k2" -> 2)
b: Map[String, Int] = Map("k2" -> 3, "k3" -> 4)
import Monoid.intAddition

m: mapMergeMonoid[String, Int] = ammonite.$sess.cmd64$Helper$mapMergeMonoid@41a9163d
res66_4: Map[String, Int] = Map("k1" -> 1, "k2" -> 5, "k3" -> 4)

#### Exercise 10.17

Write a monoid instance for functions whose results are monoids.

In [68]:
given functionMonoid[A, B](using mb: Monoid[B]): Monoid[A => B] with
    def combine(f: A => B, g: A => B): A => B = a => mb.combine(f(a), g(a))
    val empty: A => B = a => mb.empty

defined class functionMonoid

In [71]:
val fadd = summon[Monoid[Int => Int]]
fadd.combine(_ + 1, _ * 2)(10)

val fmult = functionMonoid[Int, Int](using Monoid.intMultiplication)
fmult.combine(_ + 1, _ * 2)(10)

fadd: functionMonoid[Int, Int] = ammonite.$sess.cmd67$Helper$functionMonoid@584ccb54
res70_1: Int = 31
fmult: functionMonoid[Int, Int] = ammonite.$sess.cmd67$Helper$functionMonoid@2361f813
res70_3: Int = 220

#### Exercise 10.18

A bag is like a set, except that it’s represented by a map that contains one entry per element with that element as the key, and the value under that key is the number of times the element appears in the bag. For example:

```scala
scala> bag(Vector("a", "rose", "is", "a", "rose"))
res0: Map[String,Int] = Map(a -> 2, rose -> 2, is -> 1)
```

In [72]:
def bag[A](as: IndexedSeq[A]): Map[A, Int] =
    if as.length == 0 then 
        Map()
    else if as.length == 1 then 
        Map(as(0) -> 1)
    else
        val (l, r) = as.splitAt(as.length / 2)
        mapMergeMonoid.combine(bag(l), bag(r))

defined function bag

In [73]:
bag(Vector("a", "rose", "is", "a", "rose"))

res72: Map[String, Int] = Map("a" -> 2, "rose" -> 2, "is" -> 1)

In [76]:
def bag[A](as: IndexedSeq[A]): Map[A, Int] =
    as.foldMap(a => Map(a -> 1))

defined function bag

In [77]:
bag(Vector("a", "rose", "is", "a", "rose"))

res76: Map[String, Int] = Map("a" -> 2, "rose" -> 2, "is" -> 1)